# EDA

- Permite ao cientista tomar decisões críticas sobre o que é interessante perseguir, e assim construindo uma hipótese a partir do relacionamento entre as variáveis.

## Carregamento dos dados

In [1]:
import pandas as pd

In [56]:
ks = pd.read_csv('../../../datasets/ks-projects-201801.csv')
ks = ks.sample(30000, random_state=42).reset_index().drop('index', axis=1)  # ~10%

In [3]:
from utils import *
resumetable(ks)

Dataset Shape: (30000, 15)


Name   dtypes  Missing  Uniques          First Value  \
0                 ID    int64        0    30000           1576537356   
1               name   object        0    29983                 Deko   
2           category   object        0      158             Hardware   
3      main_category   object        0       15           Technology   
4           currency   object        0       14                  USD   
5           deadline   object        0     2916           2015-10-24   
6               goal  float64        0     1531                70000   
7           launched   object        0    29997  2015-09-24 03:12:52   
8            pledged  float64        0    10421                 1888   
9              state   object        0        6               failed   
10           backers    int64        0     1223                   41   
11           country   object        0       23                   US   
12       usd pledged  float64      297    12854                 1888   
13  usd_pledged_real  float64        0    14056                 1888   
14     usd_goal_real  float64        0     7233                70000   

                                         Second Value            Third Value  \
0                                           675907016              361890770   
1   Westside BJ's: The Gluten-Free, Organic Food T...  Crepe Diem Food Truck   
2                                         Food Trucks                   Food   
3                                                Food                   Food   
4                                                 USD                    USD   
5                                          2015-02-01             2014-01-17   
6                                              250000                  30000   
7                                 2015-01-02 20:55:07    2013-12-18 03:26:04   
8                                                1466                   5723   
9                                              failed                 failed   
10                                                  9                     90   
11                                                 US                     US   
12                                               1466                   5723   
13                                               1466                   5723   
14                                             250000                  30000   

    Entropy  
0     14.87  
1     14.87  
2      6.26  
3      3.57  
4      1.28  
5     11.23  
6      6.55  
7     14.87  
8     10.72  
9      1.51  
10     6.57  
11     1.47  
12    10.86  
13    11.42  
14     8.56

## Análise inicial

- Quais colunas não são úteis se desejamos predizer o sucesso (ou não) de um projeto?
  - De cara, sabemos que `ID` e `name` são desnecessárias.
  - `category` tem 158 valores únicos, então pode ser importante para classificação.
  - `main_category` com apenas 15 valores únicos talvez não seja útil sozinho, mas em combinação com outras features, como `category` pode importar mais.
    - Igualmente para os atributos `currency` ou `country`. Um ou outro porque pode ser que estejam correlacionados, logo adicionando uma redundância desnecessária.
  - `deadline`e `launched` são do formato `datetime`, podem ser úteis para indicar alguma tendência--por exemplo, melhor mês para iniciar/terminar o projeto.
     - Também podem ser úteis para derivar dados, como `campaign_time = deadline - launched`
  - `goal` e `pledged` estão na moeda indicada por `currency`, mas os atributos `usd_pledged_real` e `usd_goal_real` indicam a mesma coisa.
     - `usd pledged` indica o mesmo que `usd_pledged_real`: convesão de `pledged` para dólar, só que feito pelo time do kickstarter. 
     - `usd_pledged_real` (e `pledged`) é quanto o projeto conseguiu arrecadar. Para predição isso é inválido pois não se sabe seu valor à priori.
     - Sendo assim, podemos dropar `goal`, `pledged`, `usd pledged` e `usd_pledged_eal`.
       - Todavia, apesar de não sabermos o valor arrecadado à priori, pode ser útil para indicar que tipos de projetos arrecadam menos.
       - O mesmo vale para `backers`, que é quantos indivíduos ajudaram.
       - Vale notar que `usd pledged` tem valores faltando.
  - `state` é o resultado final, inicialmente dividido em 6 categorias: failed, canceled, successful, suspended, live, undefined.
    - Podemos dropar os projetos `live` e `undefined`, e juntar os `canceled` e `suspended` para `failed`.
    - No fim teremos apenas `successful` e `failed`.

## Limpeza

### Dropando os mais óbvios

- São: `ID`, `name`, porque não indicam nada relevante. `goal` e `pledged` porque temos `usd_pledged_real` e `usd_goal_real`. `usd pledged` pelo mesmo motivo anterior.

In [57]:
cols2drop = ['ID', 'name', 'goal', 'pledged', 'usd pledged', 'usd_pledged_real', 'usd_goal_real']
ks.drop(cols2drop, axis=1, inplace=True)

### Agregando `state` em duas categorias

In [58]:
ks = ks.query('state != "live" and state != "undefined"')
ks['state'] = ks.state.where(ks.state == 'successful', 'failed')
ks.groupby('state').size()

state
failed        18839
successful    10691
dtype: int64

- Talvez tenhamos um problema aí: temos quase o dobro de projetos falhos em relação a projetos bem sucedidos.
- Lembre, no entanto, que estamos usando uma amostra de 10% de todo o dataset.

In [59]:
resumetable(ks)

Dataset Shape: (29530, 8)


Name  dtypes  Missing  Uniques          First Value  \
0       category  object        0      158             Hardware   
1  main_category  object        0       15           Technology   
2       currency  object        0       14                  USD   
3       deadline  object        0     2886           2015-10-24   
4       launched  object        0    29527  2015-09-24 03:12:52   
5          state  object        0        2               failed   
6        backers   int64        0     1220                   41   
7        country  object        0       23                   US   

          Second Value          Third Value  Entropy  
0          Food Trucks                 Food     6.27  
1                 Food                 Food     3.58  
2                  USD                  USD     1.27  
3           2015-02-01           2014-01-17    11.22  
4  2015-01-02 20:55:07  2013-12-18 03:26:04    14.85  
5               failed               failed     0.94  
6                    9                   90     6.61  
7                   US                   US     1.40

### Derivando dias de campanha

In [62]:
# Primeiro vamos converter as colunas com datas

ks['launched'] = pd.to_datetime(ks.launched)
ks['deadline'] = pd.to_datetime(ks.deadline)
ks['campaign_days'] = (ks.deadline - ks.launched).apply(lambda x: x.days)
ks.head(3)

category main_category currency   deadline            launched   state  \
0     Hardware    Technology      USD 2015-10-24 2015-09-24 03:12:52  failed   
1  Food Trucks          Food      USD 2015-02-01 2015-01-02 20:55:07  failed   
2         Food          Food      USD 2014-01-17 2013-12-18 03:26:04  failed   

   backers country  campaign_days  
0       41      US             29  
1        9      US             29  
2       90      US             29

- De cada data podemos obter o respectivo ano, mês e dia. São os dados que importam, e com eles podemos encontrar e visualizar possíveis relacionamento.
  - Pensando melhor: não vejo porque importaria a data fim (`deadline`) aqui. É mais útil saber a data de ínicio do projeto, dado que pode haver alguma tendência no período.
- As colunas originais podemos dropar.

In [75]:
ks = ks.assign(year=ks.launched.dt.year,
               month=ks.launched.dt.month,
               day=ks.launched.dt.day).drop(['launched', 'deadline'], axis=1)
ks.head(5)

category main_category currency   state  backers country  campaign_days  \
0     Hardware    Technology      USD  failed       41      US             29   
1  Food Trucks          Food      USD  failed        9      US             29   
2         Food          Food      USD  failed       90      US             29   
3      Fiction    Publishing      GBP  failed        2      GB             29   
4  Restaurants          Food      USD  failed        3      US             59   

   year  month  day  
0  2015      9   24  
1  2015      1    2  
2  2013     12   18  
3  2016     10   24  
4  2015      3   14

## Recursos

- https://towardsdatascience.com/hitchhikers-guide-to-exploratory-data-analysis-6e8d896d3f7e